# [Auto ML : Pycaret]

In [1]:
!pip install -q pycaret
!pip install --upgrade -q xgboost
!pip install -q catboost
!pip install scipy
! pip install holidays

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.2 MB/s e

In [2]:
import pandas as pd
import numpy as np
import holidays
import os
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from pycaret.regression import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(22) # Seed 고정

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# [0] Data Load

## [0-1] train, test, submissions

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/data/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/data/test.csv', encoding = 'utf-8')
sub = pd.read_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/data/sample_submission.csv', encoding = 'utf-8')

In [6]:
data['year'] = data['timestamp'].apply(lambda x : int(x[0:4]))
data['month'] = data['timestamp'].apply(lambda x : int(x[5:7]))
data['day'] = data['timestamp'].apply(lambda x : int(x[8:10]))

test['year'] = test['timestamp'].apply(lambda x : int(x[0:4]))
test['month'] = test['timestamp'].apply(lambda x : int(x[5:7]))
test['day'] = test['timestamp'].apply(lambda x : int(x[8:10]))

## [0-2] 사전 전처리
- 요일 추가 (월 ~ 일 : 0 ~ 6)

In [7]:
# 요일 추가
data['timestamp'] = pd.to_datetime(data['timestamp'])
#data['day_of_week'] = data['timestamp'].dt.day_name()
data['day_of_week_num'] = data['timestamp'].dt.dayofweek
data.head(10)

test['timestamp'] = pd.to_datetime(test['timestamp'])
#data['day_of_week'] = data['timestamp'].dt.day_name()
test['day_of_week_num'] = test['timestamp'].dt.dayofweek
test.head(10)

,ID,timestamp,item,corporation,location,year,month,day,day_of_week_num
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,5
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,6
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,0
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,1
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2
5,TG_A_J_20230309,2023-03-09,TG,A,J,2023,3,9,3
6,TG_A_J_20230310,2023-03-10,TG,A,J,2023,3,10,4
7,TG_A_J_20230311,2023-03-11,TG,A,J,2023,3,11,5
8,TG_A_J_20230312,2023-03-12,TG,A,J,2023,3,12,6
9,TG_A_J_20230313,2023-03-13,TG,A,J,2023,3,13,0


- 휴일 추가

In [8]:
# 한국 휴일 객체 생성
kr_holidays = holidays.KR()

# generate holiday table
holiday_df = pd.DataFrame(columns=['ds','holiday'])
holiday_df['ds'] = sorted(data['timestamp'])
holiday_df['holiday'] = holiday_df.ds.apply(lambda x: 1 if x in kr_holidays else 0)
data['holiday'] = holiday_df['holiday']

In [9]:
holiday_df = pd.DataFrame(columns=['ds','holiday'])
holiday_df['ds'] = sorted(test['timestamp'])
holiday_df['holiday'] = holiday_df.ds.apply(lambda x: 1 if x in kr_holidays else 0)
test['holiday'] = holiday_df['holiday']

- 휴일, 일요일의 price는 0으로 대체 (이상값 처리)

In [10]:
data.loc[(data['day_of_week_num'] == 6), 'price(원/kg)'] = 0
data.loc[(data['holiday'] == 1), 'price(원/kg)'] = 0

- 학습에 사용하지 않는 변수 제거

In [11]:
train_y = data['price(원/kg)']
train_x = data.drop(columns=['ID', 'timestamp', 'supply(kg)','price(원/kg)'])
test_x = test.drop(columns=['ID', 'timestamp'])

- 질적 변수들 수치화 (라벨인코더)

-> 실험 결과 원핫인코딩보다는 라벨인코더가 높은 성능을 보였다.

In [12]:
#질적 변수들을 수치화합니다
qual_col = ['item', 'corporation', 'location']

for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

print('Done.')

Done.


- OneHotEncoding

In [13]:
# qual_col = ['item', 'corporation', 'location']

# train_x = pd.get_dummies(train_x, columns=qual_col)
# test_x = pd.get_dummies(test_x, columns=qual_col)

In [14]:
train_x.head(2)

,item,corporation,location,year,month,day,day_of_week_num,holiday
0,4,0,0,2019,1,1,1,1
1,4,0,0,2019,1,2,2,1


In [15]:
test_x.head(2)

,item,corporation,location,year,month,day,day_of_week_num,holiday
0,4,0,0,2023,3,4,5,0
1,4,0,0,2023,3,5,6,0


## [0-2] trade

In [16]:
trade = pd.read_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/data/international_trade.csv', encoding = 'utf-8')
trade

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38
...,...,...,...,...,...,...,...
1269,2023-02,포포(papaw)[파파야(papaya)],0,0,23830,71,-71
1270,2023-02,사과,135165,351,0,0,351
1271,2023-02,배,2206012,5411,1,0,5411
1272,2023-02,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0


# [1] Trade 평균 피처 추가 (각 연월별 평균, 연월별평균-연도별평균, 연월별평균-월별평균)

- 주어진 trade 데이터 전체 범위의 '월별' 평균
- 주어진 과거범위 내에서 월별 평균을 한 후 칼럼을 추가하기 때문에 data leakage에 해당하지 않는다.

In [17]:
trade['year'] = trade['기간'].apply(lambda x : int(x[0:4]))
trade['month'] = trade['기간'].apply(lambda x : int(x[5:7]))

def to_season( x ):
    if x in [12, 1, 2]:
        return 0 # 겨울
    elif x in [3, 4, 5]:
        return 1 # 봄
    elif x in [6, 7, 8]:
        return 2 # 여름
    else:
        return 3 # 가을
    pass

trade['계절'] = trade['month'].apply(lambda x : to_season(x))

def to_quarter( x ):
    if x in [1,2,3]:
        return 0 # 1분기
    elif x in [4,5,6]:
        return 1 # 2분기
    elif x in [7, 8, 9]:
        return 2 # 3분기
    else:
        return 3 # 4분기
    pass

trade['분기'] = trade['month'].apply(lambda x : to_quarter(x))
trade.head(2)

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month,계절,분기
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990,2019,1,0,0
1,2019-01,양파,821330,222,4003206,1118,-896,2019,1,0,0


## [+] 연월 데이터 추가

In [18]:
trade['연월'] = trade['year'].astype(str) + '-' + trade['month'].astype(str)
train_x['연월'] = train_x['year'].astype(str) + '-' + train_x['month'].astype(str)
test_x['연월'] = test_x['year'].astype(str) + '-' + test_x['month'].astype(str)

In [19]:
trade_tmp = trade.groupby(['연월']).mean().reset_index()[['연월','수출 중량','수출 금액','수입 중량', '수입 금액', '무역수지']]
trade_tmp.columns = ["연월기준_" + col for col in trade_tmp.columns]
trade_tmp_ym = trade_tmp.rename(columns={"연월기준_연월": "연월"})
trade_tmp_ym.head(2)

,연월,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,연월기준_무역수지
0,2019-1,204860.2,471.52,1208545.52,1380.88,-909.40
1,2019-10,507261.2,795.84,985311.24,812.52,-16.68


In [20]:
trade_tmp_ym['year'] = trade_tmp_ym['연월'].apply(lambda x : int(x[0:4]))
trade_tmp_ym['month'] = trade_tmp_ym['연월'].apply(lambda x : int(x[5:7]))

In [21]:
trade_tmp_y = trade.groupby(['year']).mean().reset_index()[['year','수출 중량', '수출 금액', '수입 중량', '수입 금액', '무역수지']]
trade_tmp_y.columns = ["연기준_" + col for col in trade_tmp_y.columns]
trade_tmp_y = trade_tmp_y.rename(columns={"연기준_year": "year"})

In [22]:
trade_tmp_m = trade.groupby(['month']).mean().reset_index()[['month','수출 중량', '수출 금액', '수입 중량', '수입 금액', '무역수지']]
trade_tmp_m.columns = ["월기준_" + col for col in trade_tmp_m.columns]
trade_tmp_m = trade_tmp_m.rename(columns={"월기준_month": "month"})

In [23]:
trade_final_tmp = trade_tmp_ym.merge(trade_tmp_y, how = 'left', on = 'year').merge(trade_tmp_m, how = 'left', on = 'month')
trade_final_tmp

,연월,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,연월기준_무역수지,year,month,연기준_수출 중량,연기준_수출 금액,연기준_수입 중량,연기준_수입 금액,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지
0,2019-1,204860.200000,471.520000,1.208546e+06,1380.880000,-909.400000,2019,1,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,118959.841270,315.865079,1.000568e+06,1239.634921,-923.746032
1,2019-10,507261.200000,795.840000,9.853112e+05,812.520000,-16.680000,2019,10,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,310371.752475,689.950495,9.464158e+05,786.316832,-96.356436
2,2019-11,447252.173913,888.826087,1.134767e+06,881.478261,7.391304,2019,11,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,374065.593750,816.718750,9.046193e+05,885.791667,-69.041667
3,2019-12,289418.920000,715.680000,1.208940e+06,1003.400000,-287.720000,2019,12,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,305800.785714,698.887755,1.037158e+06,1006.734694,-307.867347
4,2019-2,79259.739130,168.173913,1.133319e+06,1376.695652,-1208.521739,2019,2,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,64412.328125,164.953125,1.027299e+06,1327.343750,-1162.359375
5,2019-3,105394.333333,205.916667,2.971708e+06,3953.208333,-3747.291667,2019,3,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,64458.708333,160.937500,3.016715e+06,4478.145833,-4317.145833
6,2019-4,317831.500000,230.625000,3.170852e+06,4776.500000,-4545.875000,2019,4,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,101978.608247,120.701031,2.449344e+06,3688.453608,-3567.783505
7,2019-5,121452.444444,126.407407,1.185548e+06,1688.518519,-1562.111111,2019,5,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,53167.842593,74.231481,1.178231e+06,1752.787037,-1678.546296
8,2019-6,551437.259259,277.518519,7.764891e+05,821.518519,-544.148148,2019,6,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,243716.560748,161.738318,8.547534e+05,1001.523364,-839.803738
9,2019-7,876870.333333,362.814815,1.058142e+06,979.777778,-617.037037,2019,7,367311.026230,450.285246,1.359267e+06,1565.409836,-1115.140984,348436.819048,211.000000,9.920146e+05,992.742857,-781.771429


In [24]:
trade_final_tmp['연기준_수출 중량'] = trade_final_tmp['연월기준_수출 중량'] - trade_final_tmp['연기준_수출 중량']
trade_final_tmp['연기준_수출 금액'] = trade_final_tmp['연월기준_수출 금액'] - trade_final_tmp['연기준_수출 금액']
trade_final_tmp['연기준_수입 중량'] = trade_final_tmp['연월기준_수입 중량'] - trade_final_tmp['연기준_수입 중량']
trade_final_tmp['연기준_수입 중량'] = trade_final_tmp['연월기준_수입 중량'] - trade_final_tmp['연기준_수입 중량']
trade_final_tmp['연기준_무역수지'] = trade_final_tmp['연월기준_무역수지'] - trade_final_tmp['연기준_무역수지']

In [25]:
trade_final_tmp['월기준_수출 중량'] = trade_final_tmp['연월기준_수출 중량'] - trade_final_tmp['월기준_수출 중량']
trade_final_tmp['월기준_수출 금액'] = trade_final_tmp['연월기준_수출 금액'] - trade_final_tmp['월기준_수출 금액']
trade_final_tmp['월기준_수입 중량'] = trade_final_tmp['연월기준_수입 중량'] - trade_final_tmp['월기준_수입 중량']
trade_final_tmp['월기준_수입 중량'] = trade_final_tmp['연월기준_수입 중량'] - trade_final_tmp['월기준_수입 중량']
trade_final_tmp['월기준_무역수지'] = trade_final_tmp['연월기준_무역수지'] - trade_final_tmp['월기준_무역수지']

In [26]:
trade_final_tmp

,연월,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,연월기준_무역수지,year,month,연기준_수출 중량,연기준_수출 금액,연기준_수입 중량,연기준_수입 금액,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지
0,2019-1,204860.200000,471.520000,1.208546e+06,1380.880000,-909.400000,2019,1,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
1,2019-10,507261.200000,795.840000,9.853112e+05,812.520000,-16.680000,2019,10,139950.173770,345.554754,1.359267e+06,1565.409836,1098.460984,196889.447525,105.889505,9.464158e+05,786.316832,79.676436
2,2019-11,447252.173913,888.826087,1.134767e+06,881.478261,7.391304,2019,11,79941.147684,438.540841,1.359267e+06,1565.409836,1122.532288,73186.580163,72.107337,9.046193e+05,885.791667,76.432971
3,2019-12,289418.920000,715.680000,1.208940e+06,1003.400000,-287.720000,2019,12,-77892.106230,265.394754,1.359267e+06,1565.409836,827.420984,-16381.865714,16.792245,1.037158e+06,1006.734694,20.147347
4,2019-2,79259.739130,168.173913,1.133319e+06,1376.695652,-1208.521739,2019,2,-288051.287099,-282.111333,1.359267e+06,1565.409836,-93.380756,14847.411005,3.220788,1.027299e+06,1327.343750,-46.162364
5,2019-3,105394.333333,205.916667,2.971708e+06,3953.208333,-3747.291667,2019,3,-261916.692896,-244.368579,1.359267e+06,1565.409836,-2632.150683,40935.625000,44.979167,3.016715e+06,4478.145833,569.854167
6,2019-4,317831.500000,230.625000,3.170852e+06,4776.500000,-4545.875000,2019,4,-49479.526230,-219.660246,1.359267e+06,1565.409836,-3430.734016,215852.891753,109.923969,2.449344e+06,3688.453608,-978.091495
7,2019-5,121452.444444,126.407407,1.185548e+06,1688.518519,-1562.111111,2019,5,-245858.581785,-323.877838,1.359267e+06,1565.409836,-446.970128,68284.601852,52.175926,1.178231e+06,1752.787037,116.435185
8,2019-6,551437.259259,277.518519,7.764891e+05,821.518519,-544.148148,2019,6,184126.233030,-172.766727,1.359267e+06,1565.409836,570.992835,307720.698512,115.780201,8.547534e+05,1001.523364,295.655590
9,2019-7,876870.333333,362.814815,1.058142e+06,979.777778,-617.037037,2019,7,509559.307104,-87.470431,1.359267e+06,1565.409836,498.103947,528433.514286,151.814815,9.920146e+05,992.742857,164.734392


In [27]:
trade_final_tmp_for_test = trade_final_tmp.groupby(['month']).mean().reset_index()
trade_final_tmp_for_test

,month,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,연월기준_무역수지,year,연기준_수출 중량,연기준_수출 금액,연기준_수입 중량,연기준_수입 금액,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지
0,1,119860.858091,318.195524,1.010879e+06,1242.307268,-924.086766,2021.0,-61232.682197,-17.532669,1.184507e+06,1483.884338,224.062468,901.016821,2.330445,1.000568e+06,1239.634921,-0.340735
1,2,64560.464017,164.724666,1.034074e+06,1329.679321,-1164.925554,2021.0,-116533.076271,-171.003527,1.184507e+06,1483.884338,-16.776320,148.135892,-0.228459,1.027299e+06,1327.343750,-2.566179
2,3,64271.302844,160.538279,3.033605e+06,4494.711721,-4334.112156,2020.5,-142109.577061,-207.221962,1.278831e+06,1566.428150,-3135.448340,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322
3,4,103028.921997,121.468924,2.469093e+06,3701.111134,-3579.674261,2020.5,-103351.957908,-246.291317,1.278831e+06,1566.428150,-2381.010446,1050.313750,0.767893,2.449344e+06,3688.453608,-11.890756
4,5,53424.878739,74.435872,1.179683e+06,1749.183532,-1674.737358,2020.5,-152956.001166,-293.324369,1.278831e+06,1566.428150,-476.073542,257.036147,0.204391,1.178231e+06,1752.787037,3.808939
5,6,244460.158908,162.247761,8.553791e+05,1001.311635,-839.082367,2020.5,38079.279003,-205.512480,1.278831e+06,1566.428150,359.581449,743.598161,0.509444,8.547534e+05,1001.523364,0.721372
6,7,344784.255698,210.409829,9.929388e+05,992.797165,-782.414373,2020.5,138403.375793,-157.350412,1.278831e+06,1566.428150,416.249442,-3652.563350,-0.590171,9.920146e+05,992.742857,-0.642945
7,8,257683.171311,505.938743,9.198430e+05,890.485013,-384.565529,2020.5,51302.291406,138.178502,1.278831e+06,1566.428150,814.098287,-1263.623081,-5.098640,9.184209e+05,892.252336,-3.331884
8,9,216010.271374,500.356319,1.052584e+06,952.927940,-452.573764,2020.5,9629.391468,132.596077,1.278831e+06,1566.428150,746.090052,-2004.547674,1.584890,1.046056e+06,946.371429,-4.973764
9,10,311563.921154,690.880000,9.446261e+05,785.775385,-94.886154,2020.5,105183.041249,323.119759,1.278831e+06,1566.428150,1103.777662,1192.168679,0.929505,9.464158e+05,786.316832,1.470282


In [28]:
trade_final_tmp = trade_final_tmp.drop(['year','month'], axis = 1)
trade_final_tmp_for_test = trade_final_tmp_for_test.drop(['year'], axis = 1)

In [29]:
train_x['연월'] = train_x['year'].astype(str) +'-'+ train_x['month'].astype(str)

In [30]:
train_x_tmp = train_x.merge(trade_final_tmp, how = 'left', on = '연월')
train_x_tmp = train_x_tmp.drop(['연월'], axis = 1)
train_x_tmp
# 3월치 데이터 3개 채워줘야 한다.

,item,corporation,location,year,month,day,day_of_week_num,holiday,연월기준_수출 중량,연월기준_수출 금액,...,연기준_수출 중량,연기준_수출 금액,연기준_수입 중량,연기준_수입 금액,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지
0,4,0,0,2019,1,1,1,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
1,4,0,0,2019,1,2,2,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
2,4,0,0,2019,1,3,3,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
3,4,0,0,2019,1,4,4,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
4,4,0,0,2019,1,5,5,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,3,5,0,2023,2,27,0,0,87940.714286,222.785714,...,7996.532468,15.185714,8.072101e+05,1153.709091,-43.766234,23528.386161,57.832589,1.027299e+06,1327.343750,172.502232
59393,3,5,0,2023,2,28,1,0,87940.714286,222.785714,...,7996.532468,15.185714,8.072101e+05,1153.709091,-43.766234,23528.386161,57.832589,1.027299e+06,1327.343750,172.502232
59394,3,5,0,2023,3,1,2,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59395,3,5,0,2023,3,2,3,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
train_x_tmp

,item,corporation,location,year,month,day,day_of_week_num,holiday,연월기준_수출 중량,연월기준_수출 금액,...,연기준_수출 중량,연기준_수출 금액,연기준_수입 중량,연기준_수입 금액,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지
0,4,0,0,2019,1,1,1,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
1,4,0,0,2019,1,2,2,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
2,4,0,0,2019,1,3,3,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
3,4,0,0,2019,1,4,4,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
4,4,0,0,2019,1,5,5,1,204860.200000,471.520000,...,-162450.826230,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,3,5,0,2023,2,27,0,0,87940.714286,222.785714,...,7996.532468,15.185714,8.072101e+05,1153.709091,-43.766234,23528.386161,57.832589,1.027299e+06,1327.343750,172.502232
59393,3,5,0,2023,2,28,1,0,87940.714286,222.785714,...,7996.532468,15.185714,8.072101e+05,1153.709091,-43.766234,23528.386161,57.832589,1.027299e+06,1327.343750,172.502232
59394,3,5,0,2023,3,1,2,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59395,3,5,0,2023,3,2,3,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
test_x_tmp = test_x.merge(trade_final_tmp_for_test, how = 'left', on = 'month')
test_x_tmp = test_x_tmp.drop(['연월'], axis = 1)
test_x_tmp

,item,corporation,location,year,month,day,day_of_week_num,holiday,연월기준_수출 중량,연월기준_수출 금액,...,연기준_수출 중량,연기준_수출 금액,연기준_수입 중량,연기준_수입 금액,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지
0,4,0,0,2023,3,4,5,0,64271.302844,160.538279,...,-142109.577061,-207.221962,1.278831e+06,1566.42815,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322
1,4,0,0,2023,3,5,6,0,64271.302844,160.538279,...,-142109.577061,-207.221962,1.278831e+06,1566.42815,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322
2,4,0,0,2023,3,6,0,0,64271.302844,160.538279,...,-142109.577061,-207.221962,1.278831e+06,1566.42815,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322
3,4,0,0,2023,3,7,1,0,64271.302844,160.538279,...,-142109.577061,-207.221962,1.278831e+06,1566.42815,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322
4,4,0,0,2023,3,8,2,0,64271.302844,160.538279,...,-142109.577061,-207.221962,1.278831e+06,1566.42815,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,3,5,0,2023,3,27,0,0,64271.302844,160.538279,...,-142109.577061,-207.221962,1.278831e+06,1566.42815,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322
1088,3,5,0,2023,3,28,1,0,64271.302844,160.538279,...,-142109.577061,-207.221962,1.278831e+06,1566.42815,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322
1089,3,5,0,2023,3,29,2,0,64271.302844,160.538279,...,-142109.577061,-207.221962,1.278831e+06,1566.42815,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322
1090,3,5,0,2023,3,30,3,0,64271.302844,160.538279,...,-142109.577061,-207.221962,1.278831e+06,1566.42815,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322


In [33]:
train_x_tmp.iloc[-3, 8:] = test_x_tmp.iloc[0,8:]
train_x_tmp.iloc[-2, 8:] = test_x_tmp.iloc[0,8:]
train_x_tmp.iloc[-1, 8:] = test_x_tmp.iloc[0,8:]

In [34]:
train_x_tmp.columns == test_x_tmp.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

# [2] 계절, 분기 추가
- 계절에 따라 거래가 없는 경우들이 있으며 가격의 변동이 있음을 EDA에서 확인했다.
- 계절 피처를 추가해 계절이 가지는 특성을 반영한다.

In [35]:
train_x_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59397 entries, 0 to 59396
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   item             59397 non-null  int64  
 1   corporation      59397 non-null  int64  
 2   location         59397 non-null  int64  
 3   year             59397 non-null  int64  
 4   month            59397 non-null  int64  
 5   day              59397 non-null  int64  
 6   day_of_week_num  59397 non-null  int64  
 7   holiday          59397 non-null  int64  
 8   연월기준_수출 중량       59283 non-null  float64
 9   연월기준_수출 금액       59283 non-null  float64
 10  연월기준_수입 중량       59283 non-null  float64
 11  연월기준_수입 금액       59283 non-null  float64
 12  연월기준_무역수지        59283 non-null  float64
 13  연기준_수출 중량        59283 non-null  float64
 14  연기준_수출 금액        59283 non-null  float64
 15  연기준_수입 중량        59283 non-null  float64
 16  연기준_수입 금액        59283 non-null  float64
 17  연기준_무역수지    

In [36]:
train_x_tmp['계절'] = train_x_tmp['month'].apply(lambda x : to_season(x))
test_x_tmp['계절'] = test_x_tmp['month'].apply(lambda x : to_season(x))

train_x_tmp

,item,corporation,location,year,month,day,day_of_week_num,holiday,연월기준_수출 중량,연월기준_수출 금액,...,연기준_수출 금액,연기준_수입 중량,연기준_수입 금액,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지,계절
0,4,0,0,2019,1,1,1,1,204860.200000,471.520000,...,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0
1,4,0,0,2019,1,2,2,1,204860.200000,471.520000,...,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0
2,4,0,0,2019,1,3,3,1,204860.200000,471.520000,...,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0
3,4,0,0,2019,1,4,4,1,204860.200000,471.520000,...,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0
4,4,0,0,2019,1,5,5,1,204860.200000,471.520000,...,21.234754,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,3,5,0,2023,2,27,0,0,87940.714286,222.785714,...,15.185714,8.072101e+05,1153.709091,-43.766234,23528.386161,57.832589,1.027299e+06,1327.343750,172.502232,0
59393,3,5,0,2023,2,28,1,0,87940.714286,222.785714,...,15.185714,8.072101e+05,1153.709091,-43.766234,23528.386161,57.832589,1.027299e+06,1327.343750,172.502232,0
59394,3,5,0,2023,3,1,2,0,64271.302844,160.538279,...,-207.221962,1.278831e+06,1566.428150,-3135.448340,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322,1
59395,3,5,0,2023,3,2,3,0,64271.302844,160.538279,...,-207.221962,1.278831e+06,1566.428150,-3135.448340,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322,1


In [37]:
train_x_tmp['분기'] = train_x_tmp['month'].apply(lambda x : to_quarter(x))
test_x_tmp['분기'] = test_x_tmp['month'].apply(lambda x : to_quarter(x))

train_x_tmp

,item,corporation,location,year,month,day,day_of_week_num,holiday,연월기준_수출 중량,연월기준_수출 금액,...,연기준_수입 중량,연기준_수입 금액,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지,계절,분기
0,4,0,0,2019,1,1,1,1,204860.200000,471.520000,...,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0,0
1,4,0,0,2019,1,2,2,1,204860.200000,471.520000,...,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0,0
2,4,0,0,2019,1,3,3,1,204860.200000,471.520000,...,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0,0
3,4,0,0,2019,1,4,4,1,204860.200000,471.520000,...,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0,0
4,4,0,0,2019,1,5,5,1,204860.200000,471.520000,...,1.359267e+06,1565.409836,205.740984,85900.358730,155.654921,1.000568e+06,1239.634921,14.346032,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,3,5,0,2023,2,27,0,0,87940.714286,222.785714,...,8.072101e+05,1153.709091,-43.766234,23528.386161,57.832589,1.027299e+06,1327.343750,172.502232,0,0
59393,3,5,0,2023,2,28,1,0,87940.714286,222.785714,...,8.072101e+05,1153.709091,-43.766234,23528.386161,57.832589,1.027299e+06,1327.343750,172.502232,0,0
59394,3,5,0,2023,3,1,2,0,64271.302844,160.538279,...,1.278831e+06,1566.428150,-3135.448340,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322,1,0
59395,3,5,0,2023,3,2,3,0,64271.302844,160.538279,...,1.278831e+06,1566.428150,-3135.448340,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322,1,0


# [3] 일요일, 휴일 여부 추가

- EDA에서 일요일과 휴일에는 금액이 0에 가까운 모습을 보였음. (극 소수의 경우를 제외하고)
- 단, 평일(월~금)은 의미있는 변화를 보이지는 않았음.
- 따라서, 일요일과 휴일을 따로 인지시킬 수 있는 칼럼 추가
(+) 현재 holiday는 1처리되어 있음
----------------------------------------------
- 일요일과 휴일일 경우 1, 평일이면 0으로 반환하는 칼럼을 생성하고, 다른 칼럼은 제거

In [38]:
train_x_tmp['holiday_sunday'] = 0
train_x_tmp.loc[(train_x_tmp['day_of_week_num'] == 6) | (train_x_tmp['holiday'] == 1), 'holiday_sunday'] = 1

In [39]:
test_x_tmp['holiday_sunday'] = 0
test_x_tmp.loc[(test_x_tmp['day_of_week_num'] == 6) | (test_x_tmp['holiday'] == 1), 'holiday_sunday'] = 1

# [4] 토요일 여부 추가
- EDA를 통해 토요일은 모든 품목의 거래금액이 소폭 감소되고 있음을 확인했다.
- 토요일 여부도 피처를 추가한다.


In [40]:
train_x_tmp['saturday'] = 0
train_x_tmp.loc[(train_x_tmp['day_of_week_num'] == 5), 'saturday'] = 1
test_x_tmp['saturday'] = 0
test_x_tmp.loc[(test_x_tmp['day_of_week_num'] == 5), 'saturday'] = 1

In [41]:
train_x_tmp = train_x_tmp.drop(['holiday', 'day_of_week_num'], axis = 1)
test_x_tmp = test_x_tmp.drop(['holiday', 'day_of_week_num'], axis = 1)

In [42]:
train_x_tmp.head(2)

,item,corporation,location,year,month,day,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,...,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지,계절,분기,holiday_sunday,saturday
0,4,0,0,2019,1,1,204860.2,471.52,1208545.52,1380.88,...,205.740984,85900.35873,155.654921,1.000568e+06,1239.634921,14.346032,0,0,1,0
1,4,0,0,2019,1,2,204860.2,471.52,1208545.52,1380.88,...,205.740984,85900.35873,155.654921,1.000568e+06,1239.634921,14.346032,0,0,1,0


In [43]:
test_x_tmp.head(2)

,item,corporation,location,year,month,day,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,...,연기준_무역수지,월기준_수출 중량,월기준_수출 금액,월기준_수입 중량,월기준_수입 금액,월기준_무역수지,계절,분기,holiday_sunday,saturday
0,4,0,0,2023,3,4,64271.302844,160.538279,3.033605e+06,4494.711721,...,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322,1,0,0,1
1,4,0,0,2023,3,5,64271.302844,160.538279,3.033605e+06,4494.711721,...,-3135.44834,-187.405489,-0.399221,3.016715e+06,4478.145833,-16.966322,1,0,1,0


# [5] Trade 계절별, 분기별 피처 추가

## [5-1] 계절

In [44]:
tmp_5 = trade.groupby(['계절']).mean().reset_index()
tmp_5 = tmp_5.drop(['year','month','분기'], axis = 1)
tmp_5.columns = ["계절별_" + col for col in tmp_5.columns]
tmp_5 = tmp_5.rename(columns={"계절별_계절": "계절"})
tmp_5

,계절,계절별_수출 중량,계절별_수출 금액,계절별_수입 중량,계절별_수입 금액,계절별_무역수지
0,0,151142.599432,367.625000,1.020475e+06,1206.687500,-839.048295
1,1,72498.631229,116.860465,2.174219e+06,3245.790698,-3128.916944
2,2,283294.184953,295.115987,9.212890e+05,961.981191,-666.887147
3,3,298507.947020,663.778146,9.677725e+05,873.586093,-209.794702


In [45]:
train_x_tmp = train_x_tmp.merge(tmp_5, on = '계절', how = 'left')
test_x_tmp = test_x_tmp.merge(tmp_5, on = '계절', how = 'left')
train_x_tmp

,item,corporation,location,year,month,day,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,...,월기준_무역수지,계절,분기,holiday_sunday,saturday,계절별_수출 중량,계절별_수출 금액,계절별_수입 중량,계절별_수입 금액,계절별_무역수지
0,4,0,0,2019,1,1,204860.200000,471.520000,1.208546e+06,1380.880000,...,14.346032,0,0,1,0,151142.599432,367.625000,1.020475e+06,1206.687500,-839.048295
1,4,0,0,2019,1,2,204860.200000,471.520000,1.208546e+06,1380.880000,...,14.346032,0,0,1,0,151142.599432,367.625000,1.020475e+06,1206.687500,-839.048295
2,4,0,0,2019,1,3,204860.200000,471.520000,1.208546e+06,1380.880000,...,14.346032,0,0,1,0,151142.599432,367.625000,1.020475e+06,1206.687500,-839.048295
3,4,0,0,2019,1,4,204860.200000,471.520000,1.208546e+06,1380.880000,...,14.346032,0,0,1,0,151142.599432,367.625000,1.020475e+06,1206.687500,-839.048295
4,4,0,0,2019,1,5,204860.200000,471.520000,1.208546e+06,1380.880000,...,14.346032,0,0,1,1,151142.599432,367.625000,1.020475e+06,1206.687500,-839.048295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,3,5,0,2023,2,27,87940.714286,222.785714,9.101820e+05,1212.714286,...,172.502232,0,0,0,0,151142.599432,367.625000,1.020475e+06,1206.687500,-839.048295
59393,3,5,0,2023,2,28,87940.714286,222.785714,9.101820e+05,1212.714286,...,172.502232,0,0,0,0,151142.599432,367.625000,1.020475e+06,1206.687500,-839.048295
59394,3,5,0,2023,3,1,64271.302844,160.538279,3.033605e+06,4494.711721,...,-16.966322,1,0,0,0,72498.631229,116.860465,2.174219e+06,3245.790698,-3128.916944
59395,3,5,0,2023,3,2,64271.302844,160.538279,3.033605e+06,4494.711721,...,-16.966322,1,0,0,0,72498.631229,116.860465,2.174219e+06,3245.790698,-3128.916944


In [46]:
train_x_tmp['계절별_수출 중량'] = train_x_tmp['계절별_수출 중량'] - train_x_tmp['연월기준_수출 중량']
train_x_tmp['계절별_수출 금액'] = train_x_tmp['계절별_수출 금액'] - train_x_tmp['연월기준_수출 금액']
train_x_tmp['계절별_수입 중량'] = train_x_tmp['계절별_수입 중량'] - train_x_tmp['연월기준_수입 중량']
train_x_tmp['계절별_수입 금액'] = train_x_tmp['계절별_수입 금액'] - train_x_tmp['연월기준_수입 금액']
train_x_tmp['계절별_무역수지'] = train_x_tmp['계절별_무역수지'] - train_x_tmp['연월기준_무역수지']

In [47]:
test_x_tmp['계절별_수출 중량'] = test_x_tmp['계절별_수출 중량'] - test_x_tmp['연월기준_수출 중량']
test_x_tmp['계절별_수출 금액'] = test_x_tmp['계절별_수출 금액'] - test_x_tmp['연월기준_수출 금액']
test_x_tmp['계절별_수입 중량'] = test_x_tmp['계절별_수입 중량'] - test_x_tmp['연월기준_수입 중량']
test_x_tmp['계절별_수입 금액'] = test_x_tmp['계절별_수입 금액'] - test_x_tmp['연월기준_수입 금액']
test_x_tmp['계절별_무역수지'] = test_x_tmp['계절별_무역수지'] - test_x_tmp['연월기준_무역수지']

## [5-2] 분기

In [48]:
tmp_5 = trade.groupby(['분기']).mean().reset_index()
tmp_5 = tmp_5.drop(['year','month','계절'], axis = 1)
tmp_5.columns = ["분기별_" + col for col in tmp_5.columns]
tmp_5 = tmp_5.rename(columns={"분기별_분기": "분기"})
tmp_5

,분기,분기별_수출 중량,분기별_수출 금액,분기별_수입 중량,분기별_수입 금액,분기별_무역수지
0,0,84062.154286,218.180000,1.563344e+06,2159.988571,-1941.771429
1,1,133691.423077,118.689103,1.462481e+06,2096.935897,-1978.259615
2,2,275030.690852,407.593060,9.850739e+05,943.463722,-535.886435
3,3,329580.749153,734.172881,9.629590e+05,891.911864,-157.732203


In [49]:
train_x_tmp = train_x_tmp.merge(tmp_5, on = '분기', how = 'left')
test_x_tmp = test_x_tmp.merge(tmp_5, on = '분기', how = 'left')
train_x_tmp

,item,corporation,location,year,month,day,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,...,계절별_수출 중량,계절별_수출 금액,계절별_수입 중량,계절별_수입 금액,계절별_무역수지,분기별_수출 중량,분기별_수출 금액,분기별_수입 중량,분기별_수입 금액,분기별_무역수지
0,4,0,0,2019,1,1,204860.200000,471.520000,1.208546e+06,1380.880000,...,-53717.600568,-103.895000,-188070.150682,-174.192500,70.351705,84062.154286,218.18,1.563344e+06,2159.988571,-1941.771429
1,4,0,0,2019,1,2,204860.200000,471.520000,1.208546e+06,1380.880000,...,-53717.600568,-103.895000,-188070.150682,-174.192500,70.351705,84062.154286,218.18,1.563344e+06,2159.988571,-1941.771429
2,4,0,0,2019,1,3,204860.200000,471.520000,1.208546e+06,1380.880000,...,-53717.600568,-103.895000,-188070.150682,-174.192500,70.351705,84062.154286,218.18,1.563344e+06,2159.988571,-1941.771429
3,4,0,0,2019,1,4,204860.200000,471.520000,1.208546e+06,1380.880000,...,-53717.600568,-103.895000,-188070.150682,-174.192500,70.351705,84062.154286,218.18,1.563344e+06,2159.988571,-1941.771429
4,4,0,0,2019,1,5,204860.200000,471.520000,1.208546e+06,1380.880000,...,-53717.600568,-103.895000,-188070.150682,-174.192500,70.351705,84062.154286,218.18,1.563344e+06,2159.988571,-1941.771429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,3,5,0,2023,2,27,87940.714286,222.785714,9.101820e+05,1212.714286,...,63201.885146,144.839286,110293.405032,-6.026786,150.808847,84062.154286,218.18,1.563344e+06,2159.988571,-1941.771429
59393,3,5,0,2023,2,28,87940.714286,222.785714,9.101820e+05,1212.714286,...,63201.885146,144.839286,110293.405032,-6.026786,150.808847,84062.154286,218.18,1.563344e+06,2159.988571,-1941.771429
59394,3,5,0,2023,3,1,64271.302844,160.538279,3.033605e+06,4494.711721,...,8227.328385,-43.677814,-859385.267144,-1248.921023,1205.195212,84062.154286,218.18,1.563344e+06,2159.988571,-1941.771429
59395,3,5,0,2023,3,2,64271.302844,160.538279,3.033605e+06,4494.711721,...,8227.328385,-43.677814,-859385.267144,-1248.921023,1205.195212,84062.154286,218.18,1.563344e+06,2159.988571,-1941.771429


In [50]:
train_x_tmp['분기별_수출 중량'] = train_x_tmp['분기별_수출 중량'] - train_x_tmp['연월기준_수출 중량']
train_x_tmp['분기별_수출 금액'] = train_x_tmp['분기별_수출 금액'] - train_x_tmp['연월기준_수출 금액']
train_x_tmp['분기별_수입 중량'] = train_x_tmp['분기별_수입 중량'] - train_x_tmp['연월기준_수입 중량']
train_x_tmp['분기별_수입 금액'] = train_x_tmp['분기별_수입 금액'] - train_x_tmp['연월기준_수입 금액']
train_x_tmp['분기별_무역수지'] = train_x_tmp['분기별_무역수지'] - train_x_tmp['연월기준_무역수지']
test_x_tmp['분기별_수출 중량'] = test_x_tmp['분기별_수출 중량'] - test_x_tmp['연월기준_수출 중량']
test_x_tmp['분기별_수출 금액'] = test_x_tmp['분기별_수출 금액'] - test_x_tmp['연월기준_수출 금액']
test_x_tmp['분기별_수입 중량'] = test_x_tmp['분기별_수입 중량'] - test_x_tmp['연월기준_수입 중량']
test_x_tmp['분기별_수입 금액'] = test_x_tmp['분기별_수입 금액'] - test_x_tmp['연월기준_수입 금액']
test_x_tmp['분기별_무역수지'] = test_x_tmp['분기별_무역수지'] - test_x_tmp['연월기준_무역수지']

# [6] 직접 연관된 trade 작물 데이터 pivot 결과 추가

- 월별 평균 사용
-  감귤, 꽃양배추와 브로콜리(broccoli), 방울다다기 양배추, 당근, 양배추  

In [51]:
trade_direct = trade[trade['품목명'].isin(['감귤', '꽃양배추와 브로콜리(broccoli)', '방울다다기 양배추', '당근', '양배추'])]
trade_direct

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month,계절,분기,연월
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,2019,1,0,0,2019-1
4,2019-01,방울다다기 양배추,0,0,7580,38,-38,2019,1,0,0,2019-1
5,2019-01,양배추,184650,94,395802,90,4,2019,1,0,0,2019-1
8,2019-01,당근,23150,22,7466150,2955,-2934,2019,1,0,0,2019-1
17,2019-01,감귤,58368,172,0,0,172,2019,1,0,0,2019-1
...,...,...,...,...,...,...,...,...,...,...,...,...
1248,2023-02,꽃양배추와 브로콜리(broccoli),24,0,332640,352,-352,2023,2,0,0,2023-2
1249,2023-02,방울다다기 양배추,71,1,10362,55,-55,2023,2,0,0,2023-2
1250,2023-02,양배추,13188,13,377456,104,-91,2023,2,0,0,2023-2
1253,2023-02,당근,22510,20,9260020,3758,-3737,2023,2,0,0,2023-2


In [52]:
trade_direct = trade_direct.groupby(['month','품목명']).mean().reset_index()

a = trade_direct.pivot(index='month',columns='품목명',values=['수출 중량']).reset_index()
a = a.droplevel(axis=1,level=0)
a.columns = ["수출중량_" + col for col in a.columns]
a = a.rename(columns={"수출중량_": "month"})

b = trade_direct.pivot(index='month',columns='품목명',values=['수출 금액']).reset_index()
b = b.droplevel(axis=1,level=0)
b.columns = ["수출금액_" + col for col in b.columns]
b = b.rename(columns={"수출금액_": "month"})

c = trade_direct.pivot(index='month',columns='품목명',values=['수입 중량']).reset_index()
c = c.droplevel(axis=1,level=0)
c.columns = ["수입중량_" + col for col in c.columns]
c = c.rename(columns={"수입중량_": "month"})

d = trade_direct.pivot(index='month',columns='품목명',values=['수입 금액']).reset_index()
d = d.droplevel(axis=1,level=0)
d.columns = ["수입금액_" + col for col in d.columns]
d = d.rename(columns={"수입금액_": "month"})

e = trade_direct.pivot(index='month',columns='품목명',values=['무역수지']).reset_index()
e = e.droplevel(axis=1,level=0)
e.columns = ["무역수지_" + col for col in e.columns]
e = e.rename(columns={"무역수지_": "month"})

trade_direct_pivot = a.merge(b, how = 'left', on = 'month').merge(c, how = 'left', on = 'month').merge(d, how = 'left', on = 'month').merge(e, how = 'left', on = 'month')
trade_direct_pivot = trade_direct_pivot.fillna(0)
trade_direct_pivot

,month,수출중량_감귤,수출중량_꽃양배추와 브로콜리(broccoli),수출중량_당근,수출중량_방울다다기 양배추,수출중량_양배추,수출금액_감귤,수출금액_꽃양배추와 브로콜리(broccoli),수출금액_당근,수출금액_방울다다기 양배추,...,수입금액_감귤,수입금액_꽃양배추와 브로콜리(broccoli),수입금액_당근,수입금액_방울다다기 양배추,수입금액_양배추,무역수지_감귤,무역수지_꽃양배추와 브로콜리(broccoli),무역수지_당근,무역수지_방울다다기 양배추,무역수지_양배추
0,1,192621.60,46.40,22897.40,0.0,103523.60,317.40,0.40,29.40,0.0,...,0.00,751.80,2758.00,72.40,335.00,317.40,-751.20,-2729.00,-72.40,-277.20
1,2,23706.40,423.20,17524.00,14.2,61673.00,112.00,1.00,17.20,0.2,...,51.40,432.80,2829.00,54.40,199.60,60.60,-431.80,-2812.20,-54.40,-169.00
2,3,7207.50,45.50,17275.00,0.0,219985.50,33.75,1.00,17.25,0.0,...,14.50,819.50,4787.75,54.00,327.00,19.25,-818.50,-4770.50,-54.00,-231.50
3,4,940.25,33.50,18235.50,0.0,1642495.75,5.50,0.25,16.00,0.0,...,44.50,1106.00,3879.25,96.75,566.75,-39.00,-1105.75,-3863.00,-96.75,112.00
4,5,2010.50,18.00,15800.00,0.0,572786.50,18.00,0.25,14.50,0.0,...,19.25,856.25,2617.25,120.25,227.00,-1.25,-855.75,-2603.00,-120.25,42.75
5,6,3760.50,12.00,12659.00,0.0,1859007.50,28.25,0.00,9.75,0.0,...,0.00,569.00,4005.50,128.25,100.75,28.25,-569.00,-3996.00,-128.25,813.25
6,7,7866.75,20.00,19842.00,0.0,1453352.50,48.75,0.00,16.00,0.0,...,0.00,744.00,3711.50,125.75,104.25,48.75,-744.00,-3695.75,-125.75,609.75
7,8,7344.00,8.25,16103.25,0.0,1283834.75,44.00,0.00,12.75,0.0,...,0.00,1466.75,2634.25,139.50,202.75,44.00,-1466.75,-2622.00,-139.50,469.00
8,9,11046.75,6.00,17378.75,0.0,765960.00,59.75,0.00,13.25,0.0,...,0.00,1677.00,4239.25,108.00,319.75,59.75,-1677.00,-4226.50,-108.00,102.50
9,10,81898.50,6.00,17047.00,0.0,126470.75,95.50,0.00,15.00,0.0,...,0.00,1081.00,3379.75,105.25,529.75,95.50,-1081.00,-3364.75,-105.25,-450.50


In [53]:
trade_direct_pivot['month'] = trade_direct_pivot['month'].astype(int) # merge를 위한 형변환

In [54]:
train_x_tmp = train_x_tmp.merge(trade_direct_pivot, how = 'left', on = 'month')
test_x_tmp = test_x_tmp.merge(trade_direct_pivot, how = 'left', on = 'month')

## [+] 최종 데이터 확인

In [55]:
train_x_tmp

,item,corporation,location,year,month,day,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,...,수입금액_감귤,수입금액_꽃양배추와 브로콜리(broccoli),수입금액_당근,수입금액_방울다다기 양배추,수입금액_양배추,무역수지_감귤,무역수지_꽃양배추와 브로콜리(broccoli),무역수지_당근,무역수지_방울다다기 양배추,무역수지_양배추
0,4,0,0,2019,1,1,204860.200000,471.520000,1.208546e+06,1380.880000,...,0.0,751.8,2758.00,72.4,335.0,317.40,-751.2,-2729.0,-72.4,-277.2
1,4,0,0,2019,1,2,204860.200000,471.520000,1.208546e+06,1380.880000,...,0.0,751.8,2758.00,72.4,335.0,317.40,-751.2,-2729.0,-72.4,-277.2
2,4,0,0,2019,1,3,204860.200000,471.520000,1.208546e+06,1380.880000,...,0.0,751.8,2758.00,72.4,335.0,317.40,-751.2,-2729.0,-72.4,-277.2
3,4,0,0,2019,1,4,204860.200000,471.520000,1.208546e+06,1380.880000,...,0.0,751.8,2758.00,72.4,335.0,317.40,-751.2,-2729.0,-72.4,-277.2
4,4,0,0,2019,1,5,204860.200000,471.520000,1.208546e+06,1380.880000,...,0.0,751.8,2758.00,72.4,335.0,317.40,-751.2,-2729.0,-72.4,-277.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,3,5,0,2023,2,27,87940.714286,222.785714,9.101820e+05,1212.714286,...,51.4,432.8,2829.00,54.4,199.6,60.60,-431.8,-2812.2,-54.4,-169.0
59393,3,5,0,2023,2,28,87940.714286,222.785714,9.101820e+05,1212.714286,...,51.4,432.8,2829.00,54.4,199.6,60.60,-431.8,-2812.2,-54.4,-169.0
59394,3,5,0,2023,3,1,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5
59395,3,5,0,2023,3,2,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5


In [56]:
test_x_tmp

,item,corporation,location,year,month,day,연월기준_수출 중량,연월기준_수출 금액,연월기준_수입 중량,연월기준_수입 금액,...,수입금액_감귤,수입금액_꽃양배추와 브로콜리(broccoli),수입금액_당근,수입금액_방울다다기 양배추,수입금액_양배추,무역수지_감귤,무역수지_꽃양배추와 브로콜리(broccoli),무역수지_당근,무역수지_방울다다기 양배추,무역수지_양배추
0,4,0,0,2023,3,4,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5
1,4,0,0,2023,3,5,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5
2,4,0,0,2023,3,6,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5
3,4,0,0,2023,3,7,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5
4,4,0,0,2023,3,8,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,3,5,0,2023,3,27,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5
1088,3,5,0,2023,3,28,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5
1089,3,5,0,2023,3,29,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5
1090,3,5,0,2023,3,30,64271.302844,160.538279,3.033605e+06,4494.711721,...,14.5,819.5,4787.75,54.0,327.0,19.25,-818.5,-4770.5,-54.0,-231.5


# [+] 모델링

In [57]:
X_train, X_val, y_train, y_val = train_test_split(train_x_tmp, train_y, random_state = 7, test_size=0.2)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47517, 60), (11880, 60), (47517,), (11880,))

In [58]:
X = pd.concat([X_train, y_train], axis = 1)
X.shape

(47517, 61)

- Pycaret setup (기초 설정)

In [59]:
reg = setup(data = X # 데이터
      ,target = 'price(원/kg)' # Target
      ,verbose = True # 진행 로그 출력
      ,train_size = 0.7 # 내부적으로 검증시 훈련데이터의 비율
      ,data_split_shuffle = True # 데이터 split시 데이터를 섞을것인지 여부
      ,normalize = True # 정규화 여부
      ,session_id = 100 # 난수 시드
      )

,Description,Value
0,Session id,100
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(47517, 61)"
4,Transformed data shape,"(47517, 61)"
5,Transformed train set shape,"(33261, 61)"
6,Transformed test set shape,"(14256, 61)"
7,Numeric features,60
8,Rows with missing values,0.2%
9,Preprocess,True


- 사용 가능한 모델 확인

In [60]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


- 전체 모델 중, 성능이 훌륭했던 random forest, catboost, lightgbm, xgboost로 학습한다.

In [61]:
## 실행중 ##
model_rf = create_model('rf', fold = 10)
model_cb = create_model('catboost', fold = 10)
model_lgb = create_model('lightgbm', fold = 10)
model_xgb = create_model('xgboost', fold = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,315.3291,823778.6897,907.6225,0.7996,2.1043,0.2072
1,305.9079,785023.1553,886.0153,0.8185,2.0688,0.1871
2,324.4255,905512.7137,951.5843,0.7966,2.1614,0.1855
3,276.8195,692047.8797,831.8942,0.8203,2.0519,0.1842
4,291.4085,751206.6852,866.7218,0.8068,2.1170,0.1907
5,289.0225,688665.1492,829.8585,0.8310,2.0690,0.1942
6,283.3542,691483.0496,831.5546,0.8282,2.0520,0.1874
7,279.1340,652264.9692,807.6292,0.8453,2.0244,0.1917
8,279.7765,708963.6653,841.9998,0.8144,2.1086,0.1899


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,420.4071,808959.8025,899.4219,0.8032,3.8202,0.2534
1,410.9115,788061.4611,887.7283,0.8178,3.7158,0.2422
2,426.0823,868503.4799,931.9353,0.8049,3.8336,0.2426
3,382.0644,710474.7525,842.8966,0.8155,3.8061,0.2433
4,383.2771,693171.3750,832.5691,0.8217,3.8198,0.2380
5,399.8113,712437.6539,844.0602,0.8252,3.7964,0.2499
6,394.2862,710307.8784,842.7976,0.8236,3.8143,0.2507
7,394.2487,699517.3229,836.3715,0.8341,3.7953,0.2449
8,389.0295,740691.7818,860.6345,0.8061,3.8639,0.2469


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,406.1866,814793.1768,902.6589,0.8017,3.5313,0.2713
1,400.7787,818352.4898,904.6284,0.8108,3.4373,0.2553
2,408.1800,875630.7575,935.7514,0.8033,3.4951,0.2581
3,364.0071,698846.3042,835.9703,0.8185,3.4924,0.2546
4,367.5085,690735.9191,831.1052,0.8223,3.5057,0.2565
5,383.6322,717633.5111,847.1325,0.8239,3.4867,0.2707
6,376.1664,724885.8253,851.4023,0.8199,3.4827,0.2584
7,376.6561,712825.3012,844.2898,0.8309,3.4592,0.2550
8,380.4862,778488.9017,882.3202,0.7962,3.5462,0.2737


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,425.5265,822473.9375,906.9035,0.7999,3.8217,0.2565
1,414.9018,818463.8750,904.6899,0.8107,3.7440,0.2380
2,428.5653,899776.2500,948.5654,0.7979,3.8312,0.2464
3,378.4729,718139.7500,847.4313,0.8135,3.8199,0.2407
4,380.8828,689736.6875,830.5038,0.8226,3.8378,0.2382
5,390.3832,685659.0000,828.0453,0.8317,3.7666,0.2445
6,393.0982,730222.0625,854.5303,0.8186,3.7494,0.2533
7,390.9753,710696.0000,843.0279,0.8314,3.7723,0.2463
8,387.1094,739740.5625,860.0817,0.8063,3.8712,0.2460


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

- 랜덤포레스트를 제외한 나머지 모델들을 튜닝한다.

In [62]:
#tune_model_rf = tune_model(model_rf, n_iter = 50, fold = 5, optimize = 'mse', choose_better = True)
tune_model_cb = tune_model(model_cb, n_iter = 50, fold = 5, optimize = 'mse', choose_better = True)
tune_model_lgb = tune_model(model_lgb, n_iter = 50, fold = 5, optimize = 'mse', choose_better = True)
tune_model_xgb = tune_model(model_xgb, n_iter = 50, fold = 5, optimize = 'mse', choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,383.4203,804485.8854,896.9314,0.8093,3.5295,0.2381
1,373.1427,793377.9440,890.7177,0.8090,3.5715,0.2317
2,361.1394,694286.4029,833.2385,0.8256,3.5647,0.2365
3,357.1498,665647.3359,815.8721,0.8385,3.5430,0.2395
4,360.2883,734341.9334,856.9375,0.8016,3.6093,0.2371
Mean,367.0281,738427.9003,858.7394,0.8168,3.5636,0.2366
Std,9.8364,54120.8688,31.5353,0.0134,0.0274,0.0026


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 50 candidates, totalling 250 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,387.5761,821537.7884,906.3872,0.8052,3.3391,0.2299
1,372.2578,794456.8957,891.3231,0.8088,3.4029,0.2192
2,358.1856,717935.9388,847.3110,0.8197,3.3418,0.2276
3,362.3320,723086.1623,850.3447,0.8245,3.3411,0.2283
4,362.1860,757656.9504,870.4349,0.7953,3.4051,0.2306
Mean,368.5075,762934.7471,873.1602,0.8107,3.3660,0.2271
Std,10.6049,40171.1835,22.9350,0.0104,0.0311,0.0041


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 50 candidates, totalling 250 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,355.6196,797748.5625,893.1677,0.8109,3.2255,0.2138
1,344.5228,769267.0000,877.0787,0.8148,3.2681,0.2099
2,336.7740,703492.6250,838.7447,0.8233,3.2540,0.2190
3,331.8130,658784.7500,811.6556,0.8401,3.2288,0.2139
4,337.2078,723061.0625,850.3300,0.8046,3.3087,0.2174
Mean,341.1874,730470.8000,854.1953,0.8188,3.2570,0.2148
Std,8.2778,48893.8817,28.6559,0.0123,0.0303,0.0032


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 50 candidates, totalling 250 fits


- 스태킹 모델 후보 선정

In [63]:
candidate_models = [tune_model_cb, tune_model_lgb, tune_model_xgb, model_rf]

- 스태킹 모델 구축

In [ ]:
stacker_false = stack_models(candidate_models, fold = 10, choose_better = True, restack = False)

In [ ]:
stack_model = finalize_model(stacker_false)
stack_model

- 최종 예측 수행

In [ ]:
pred_test = predict_model(stack_model, data = test_x_tmp)
pred_test

### [+] 결과물 후처리

- 일요일은 0으로 처리한다.

In [ ]:
sum(pred_test.prediction_label < 0)

In [ ]:
test['timestamp'] = pd.to_datetime(test['timestamp'])
test['day_of_week'] = test['timestamp'].dt.day_name()
test['day_of_week_num'] = test['timestamp'].dt.dayofweek
test.head(10)

In [ ]:
pred_test['week'] = test['day_of_week']

In [ ]:
pred_test.loc[pred_test['week'] == 'Sunday', 'prediction_label'] = 0

In [ ]:
sum(pred_test.prediction_label < 0)

- 그럼에도 불구하고 0보다 작은 값들은 품목별, 요일별 평균으로 대체한다.
- 3월은 모든 품목이 거래가 일반적으로 존재한다. 따라서 휴일과 일요일을 제외한다면 거래가 존재하는것이 일반적이다.

In [ ]:
tmp_for_pred = pred_test.groupby(['item','week']).mean().reset_index()[['item','week','prediction_label']]

In [ ]:
idx_under_zero = pred_test['prediction_label'] < 0
pred_test.loc[idx_under_zero, 'prediction_label'] = pred_test.groupby(['item', 'week'])['prediction_label'].transform('mean')

In [ ]:
sub['answer_ML'] = pred_test['prediction_label']
sub

In [ ]:
sub.to_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/submissions/final_try_only_ML.csv', index=False)